In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
from PIL import Image 
import PIL 

In [ ]:
#perform GaussianBlur & Canny algorithm
def find_edge(file,save_path):
    image = mpimg.imread(file)
    
    image=(image*255).astype(np.uint8)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kernel_size = 9
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size), 0)
    low_threshold = 60
    high_threshold = 100
    edges = cv2.Canny(gray, low_threshold, high_threshold)
    
    cv2.imwrite(save_path, edges)

In [ ]:
#go through all image(find_edge)
src_dir = r''    #source directory
dest_dir = r''         #destination directory
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)
for child_dir in os.listdir(src_dir):
    new_name = os.path.join(dest_dir,child_dir.split('.')[0] + '.png')
    new_name = os.path.join(dest_dir,child_dir.split('.')[0] + '.png')
    old_mask = os.path.join(os.path.join(src_dir, child_dir))
    find_edge(old_mask,new_name)
    

In [ ]:
#past edge image on mask graph
def combine_edge_mask(edge,mask,save_path):
    #turning mask graph from bgr to rgb
    image = mpimg.imread(mask)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    #turning edge graph from gray_scale to rgb
    image2 = mpimg.imread(edge)
    image2 = cv2.cvtColor(image2, cv2.COLOR_GRAY2RGB)
    
    for i,j in zip(np.where(image!=0)[0],np.where(image!=0)[1]):
        if(not(image2[i][j].all(0))):   #take this out the polyp will not have edge
            image2[i][j]=image[i][j]

    image2=(image2*255).astype(np.uint8)
    cv2.imwrite(save_path,image2)
    #imgplot = plt.imshow(image2)      
    #plt.show()

In [ ]:
#go through all image(combine_edge_mask)
bot_dir = r''   #mask directory
top_dir = r''   #edge directory
combine = r''   #destination directory
if not os.path.exists(combine):
    os.makedirs(combine)
for child_dir in os.listdir(top_dir):
    edge = os.path.join(os.path.join(top_dir, child_dir))
    mask = os.path.join(os.path.join(bot_dir, child_dir))
    new_name = os.path.join(combine,child_dir.split('.')[0] + '.png')
    combine_edge_mask(edge,mask,new_name)

In [ ]:
#read the output of combine_edge_mask and original image and generate an image with original background and the mask&edge polyp
#this image will use as a condtion in Conditional GAN
def combine_edge_mask2(mask,ori_combine,ori_img,save_path):
    #change bgr to rgb
    image = mpimg.imread(mask)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    image3 = mpimg.imread(ori_combine)
    image3 = cv2.cvtColor(image3, cv2.COLOR_BGR2RGB)
    
    image4 = mpimg.imread(ori_img)
    image4 = cv2.cvtColor(image4, cv2.COLOR_BGR2RGB)
    
    #past it
    for i,j in zip(np.where(image!=0)[0],np.where(image!=0)[1]):
        image4[i][j]=image3[i][j]    

    image4=(image4*255).astype(np.uint8)
    cv2.imwrite(save_path,image4)
    #imgplot = plt.imshow(image2)
    #plt.show()

In [ ]:
bot_dir = r''       #mask directory
top_dir = r''       #combine_edge_mask directory
ori_dir = r''       #original image directory
combine = r''       #destination directory
if not os.path.exists(combine):
    os.makedirs(combine)
for child_dir in os.listdir(top_dir):
    mask = os.path.join(os.path.join(bot_dir, child_dir))
    ori_combine = os.path.join(os.path.join(top_dir, child_dir))
    ori_img = os.path.join(os.path.join(ori_dir, child_dir))
    new_name = os.path.join(combine,child_dir.split('.')[0] + '.png')
    combine_edge_mask2(mask,ori_combine,ori_img,new_name)


In [ ]:
#past the condition image on the right of original image
#the conditional project will read the left image as target image and the right as condtional
combine_dir = r''          #combine_edge_mask2 directory
real_dir = r''             #original directory
train_file = r''           #destination directory

for child_dir in os.listdir(combine_dir):
    combine = mpimg.imread(os.path.join(combine_dir, child_dir))
    real = mpimg.imread(os.path.join(real_dir, child_dir))
    save_path = os.path.join(train_file, child_dir)
    image_h = cv2.hconcat([real, combine])
    image_h=(image_h*255).astype(np.uint8)
    image_h=cv2.cvtColor(image_h, cv2.COLOR_BGR2RGB)
    #imgplot = plt.imshow(image_h)
    #plt.show()
    cv2.imwrite(save_path.replace(".png",".jpg"),image_h)